# Interactive Financial AI Advisor

This notebook allows you to interact with and iterate on the Financial AI Advisor system.

## Features
- Test different advisor personas
- Compare prompt variations
- Experiment with scenarios
- Visualize responses
- Iterate quickly without API


In [ ]:
# Add parent directory to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

# Standard imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import json

# Load environment variables
load_dotenv('../.env')

print("✅ Environment loaded")
print(f"OpenAI Key present: {bool(os.getenv('OPENAI_API_KEY'))}")


✅ Environment loaded
OpenAI Key present: True
Key starts with: sk-proj-x9fjt4sxuSJw...


In [2]:
# Quick API Test
from openai import OpenAI

try:
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Say 'API key works!' in 5 words"}],
        max_tokens=20
    )
    print("✅ API Key is VALID and working!")
    print(f"📝 Test response: {response.choices[0].message.content}")
except Exception as e:
    print(f"❌ API Key ERROR: {e}")
    print("⚠️  Please check your API key and try again")


✅ API Key is VALID and working!
📝 Test response: API key functions successfully, hooray!


In [3]:
# Import our advisor classes
from src.advisors.cfp_advisor import CFPAdvisor, CFPBasicAdvisor, CFPExpertiseAdvisor
from src.models.client_profile import (
    FinancialScenario,
    ClientProfile,
    IncomeInfo,
    ExpensesInfo,
    DebtInfo,
    AssetsInfo,
    FinancialGoal,
    LifeStage,
)

print("✅ Imports successful")


✅ Imports successful


/opt/anaconda3/envs/financial-advisor/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_used" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## 1. Quick Test - Simple Scenario

Start with a simple scenario to test everything works.


In [4]:
# Create a simple client profile
simple_profile = ClientProfile(
    age=28,
    income=IncomeInfo(annual_salary=95000),
    expenses=ExpensesInfo(
        housing=1400,
        food=500,
        transportation=400,
        other=1900,
    ),
    debts=DebtInfo(credit_card=8000),
    assets=AssetsInfo(checking_savings=2000),
)

# Create scenario
simple_scenario = FinancialScenario(
    client_profile=simple_profile,
    windfall_amount=15000,
    question="I just received a $15,000 bonus. What should I do with it?",
)

# Display scenario summary
print("📊 Client Profile Summary")
print(f"Age: {simple_profile.age}")
print(f"Income: ${simple_profile.income.total_annual:,.0f}/year")
print(f"Expenses: ${simple_profile.expenses.total_monthly:,.0f}/month")
print(f"Debt: ${simple_profile.debts.total:,.0f}")
print(f"Savings: ${simple_profile.assets.checking_savings:,.0f}")
print(f"Windfall: ${simple_scenario.windfall_amount:,.0f}")
print(f"\n❓ Question: {simple_scenario.question}")


📊 Client Profile Summary
Age: 28
Income: $95,000/year
Expenses: $4,200/month
Debt: $8,000
Savings: $2,000
Windfall: $15,000

❓ Question: I just received a $15,000 bonus. What should I do with it?


In [ ]:
# Get advice from CFP advisor
advisor = CFPAdvisor(
    model="gpt-4o-mini",  # Use mini for cost-effective testing
    temperature=0.7,
    api_key=os.getenv("OPENAI_API_KEY"),
)

print("🤔 Generating advice...\n")
advice = advisor.provide_advice(simple_scenario)

# Display advice
display(Markdown(f"## Financial Advice\n\n{advice.summary}"))


🤔 Generating advice...



## Financial Advice

Based on your financial situation, here are prioritized actions to consider for your $15,000 bonus, ensuring you address both immediate concerns and long-term wealth building.

### 1. PRIORITY ACTIONS

**1. Pay Off Credit Card Debt**: $8,000  
**2. Build Emergency Fund**: $5,000  
**3. Contribute to Retirement Account (401(k) or IRA)**: $2,000  

### 2. DETAILED REASONING

**1. Pay Off Credit Card Debt ($8,000)**  
Credit card debt typically carries high interest rates, often exceeding 20%. By paying off this debt, you eliminate a significant financial burden and free up your monthly cash flow. This will save you money in interest payments over time.

**2. Build Emergency Fund ($5,000)**  
Your current emergency savings of $2,000 is below the recommended 3-6 months of expenses ($12,600 - $25,200 based on your $4,200 monthly expenses). By adding $5,000, you'll have a total of $7,000, which provides a more substantial cushion against unexpected expenses, such as medical bills or job loss.

**3. Contribute to Retirement Account ($2,000)**  
Investing in your retirement early can lead to significant growth due to compound interest. Contributing $2,000 to a 401(k) or IRA can help you maximize your tax-advantaged retirement savings, especially if your employer matches contributions to your 401(k).

### 3. TIMELINE

- **Immediate (Within 1 Month)**: Pay off the credit card debt of $8,000.
- **Next 2-3 Months**: Allocate $5,000 to your emergency fund. Consider setting up automatic transfers to a high-yield savings account.
- **By the End of the Year**: Contribute $2,000 to your retirement account, taking advantage of any employer match that may be available.

### 4. EXPECTED OUTCOMES

- **Debt-Free**: Paying off your credit card will save you approximately $1,600 in interest over the next two years, assuming a 20% APR.
- **Stronger Financial Safety Net**: With a total of $7,000 in your emergency fund, you'll be closer to having 2 months’ worth of expenses saved, significantly reducing financial anxiety.
- **Retirement Growth**: The $2,000 invested today could grow to approximately $10,000 in 30 years assuming an average annual return of 7%, providing you with a solid foundation for retirement.

### 5. NEXT STEPS

1. **Pay Off the Credit Card**: Contact your credit card company to confirm the payoff amount and make the payment.
2. **Set Up Emergency Fund**: Identify a high-yield savings account. Transfer the $5,000 and set up automatic contributions to gradually build up to your target.
3. **Retirement Contribution**: Review your employer's retirement plan and consider increasing your contributions. If your employer matches, try to contribute enough to receive the full match.

**Follow-up Recommendations**: Schedule a check-in in 6 months to review your progress on debt, savings, and investments. This will allow us to adjust your financial plan as your situation and goals evolve.

## 2. Compare Different Personas

See how different prompt engineering approaches affect the quality of advice.


In [6]:
# Test all three persona variations
personas = [
    ("Basic Advisor", CFPBasicAdvisor),
    ("CFP with Expertise", CFPExpertiseAdvisor),
    ("CFP Full (with Style)", CFPAdvisor),
]

responses = {}

for name, AdvisorClass in personas:
    print(f"\n{'='*80}")
    print(f"🎭 {name}")
    print(f"{'='*80}\n")
    
    advisor = AdvisorClass(
        model="gpt-4o-mini",
        temperature=0.7,
        api_key=os.getenv("OPENAI_API_KEY"),
    )
    
    advice = advisor.provide_advice(simple_scenario)
    responses[name] = advice
    
    display(Markdown(advice.summary))
    print(f"\n[Model: {advice.model_used} | Persona: {advice.persona}]\n")



🎭 Basic Advisor



Congratulations on your bonus! With your current financial situation in mind, let’s outline a strategic approach to allocate that $15,000 effectively.

### 1. Pay Down Debt
**Credit Card Debt: $8,000**
- **Recommendation**: Pay down your credit card debt in full. This is a high-interest debt that can significantly hinder your financial progress. Paying it off will not only relieve you from monthly interest payments but also improve your credit score.

### 2. Build Emergency Savings
**Current Emergency Savings: $2,000**
- **Recommendation**: Aim to have 3 to 6 months’ worth of living expenses saved for emergencies. Your monthly expenses are approximately $4,200, which means you should ideally have between $12,600 and $25,200 in savings.
- **Action**: Consider allocating $5,000 to your emergency fund. This increases your total savings to $7,000, which is a good start. You can continue to build this fund over time.

### 3. Invest in Your Future
**Retirement Savings**
- **Recommendation**: If you have a 401(k) or similar retirement plan at work, consider contributing a portion of your bonus to this account, especially if your employer matches contributions. A good target could be to contribute around $2,000.
- **Action**: This will not only help you build your retirement savings but also reduce your taxable income for the year.

### 4. Short-Term Goals
- **Consider Allocating Remaining Funds**: After paying off the credit card ($8,000) and contributing to your emergency fund ($5,000), you'll have $2,000 remaining.
- **Recommendation**: Think about what short-term goals you have. This could include further education, travel, or even saving for a down payment on a home. Allocate this remaining amount towards a specific goal that aligns with your personal aspirations.

### Summary of Allocation:
- **Pay Off Credit Card Debt**: $8,000
- **Emergency Savings**: $5,000
- **Retirement Savings (401k)**: $2,000
- **Short-Term Goals or Miscellaneous**: $0 (You may not have any funds left after the above allocations, but if you can adjust your emergency fund or retirement contributions, consider keeping some liquid for flexibility.)

### Additional Considerations:
- **Review Budget**: Post these changes, review your budget to ensure you can comfortably manage your monthly expenses and continue building your savings.
- **Set Financial Goals**: Think about your financial goals for the next few years, whether it be home ownership, further education, or travel, and start planning how to allocate funds toward these objectives.

By taking these steps, you’ll be in a stronger financial position, reduce your debt burden, and lay the groundwork for future wealth accumulation.


[Model: gpt-4o-mini | Persona: CFP (Basic)]


🎭 CFP with Expertise



Congratulations on your bonus! Here’s a step-by-step approach to optimize the use of your $15,000 windfall, considering your current financial situation:

### 1. **Build Your Emergency Fund**
Given that you only have $2,000 saved for emergencies and your monthly expenses are around $4,200, it would be prudent to increase your emergency savings. Aiming for at least 3-6 months' worth of expenses is ideal, which means you should target an emergency fund of approximately $12,600 to $25,200.

**Recommendation: Allocate $5,000 to your emergency fund.**

### 2. **Pay Off High-Interest Debt**
Your credit card debt of $8,000 likely comes with a high-interest rate. Paying this off can save you significant interest costs over time and improve your cash flow.

**Recommendation: Use $8,000 to pay off the credit card debt.**

### 3. **Contribute to Retirement Accounts**
Since you're 28, time is on your side when it comes to retirement savings. Consider maximizing contributions to your 401(k) or IRA, especially if your employer offers a match, as this is essentially free money.

**Recommendation: Allocate $2,000 to your 401(k) or IRA.** Aim to contribute enough to get any employer match if available.

### 4. **Invest for Future Goals**
If you have other financial goals, such as saving for a home or investing for long-term growth, consider using any remaining funds to build a diversified investment portfolio. 

**Recommendation: With the remaining $0 (after the above allocations), consider starting a brokerage account or a 529 plan if you plan on funding education in the future.**

### Summary of Allocations:
- **Emergency Fund:** $5,000
- **Credit Card Debt Payoff:** $8,000
- **Retirement Account Contribution:** $2,000

### Final Thoughts:
After these allocations, you will have a stronger financial foundation with an emergency fund, no credit card debt, and increased retirement savings. Make sure to reassess your budget to accommodate any changes in your monthly expenses due to the debt payoff and new savings habits. Additionally, consider adjusting your cash flow management strategies to ensure you continue to build wealth systematically moving forward.


[Model: gpt-4o-mini | Persona: CFP (Expertise Enhanced)]


🎭 CFP Full (with Style)



Certainly! Here’s a structured financial action plan tailored for your situation:

### 1. PRIORITY ACTIONS

**1. Pay Down Credit Card Debt: $8,000**  
**2. Build Emergency Fund: $5,000**  
**3. Invest for the Future: $2,000**  

### 2. DETAILED REASONING

**1. Pay Down Credit Card Debt: $8,000**  
Credit card debt typically carries high interest rates, often exceeding 20%. By paying off your $8,000 credit card balance, you effectively save yourself from accruing high interest, which can lead to significant financial strain over time. Eliminating this debt will also improve your credit score, which is beneficial for future financial endeavors like purchasing a home.

**2. Build Emergency Fund: $5,000**  
An emergency fund should ideally cover 3-6 months of living expenses. Given your monthly expenses of approximately $4,200, a minimum emergency fund of $12,600 is recommended. With your current savings of $2,000, adding $5,000 from your bonus will bring your total to $7,000. This provides a solid foundation and reduces financial stress in case of unexpected expenses.

**3. Invest for the Future: $2,000**  
After addressing immediate debts and building your emergency savings, it's vital to start investing for wealth accumulation. You can consider contributing $2,000 to a tax-advantaged retirement account like a Roth IRA or a brokerage account. This will help you take advantage of compound interest over time, ensuring your money grows.

### 3. TIMELINE

- **Immediate (1-2 weeks)**: Pay off the credit card debt of $8,000.
- **Short-term (1 month)**: Transfer $5,000 to a high-yield savings account for your emergency fund.
- **Medium-term (3 months)**: Invest $2,000 in a suitable investment account, focusing on a diversified portfolio.

### 4. EXPECTED OUTCOMES

- **Debt-Free**: Eliminating your credit card debt will save you from high-interest payments and improve your credit score, potentially increasing it by 20-30 points.
- **Emergency Fund**: With $7,000 saved, you’ll be closer to having a secure financial cushion, covering approximately 1.5 months of expenses.
- **Investment Growth**: Starting with $2,000 in investments can lead to significant growth over time, depending on market conditions, with potential returns of 5-7% annually.

### 5. NEXT STEPS

1. **Pay Off Credit Card**: Contact your credit card provider to ensure the balance is settled and confirm zero outstanding debt.
2. **Open High-Yield Savings Account**: Research and select a reputable bank offering competitive interest rates for your emergency fund.
3. **Research Investment Options**: Look into opening a Roth IRA or a brokerage account. Consider a diversified ETF or mutual fund to start, focusing on long-term growth.

After completing these steps, we can schedule a follow-up appointment to review your progress and discuss further investment strategies or financial goals.


[Model: gpt-4o-mini | Persona: Certified Financial Planner (CFP)]



## 3. Quick Reference - Fast Iteration

Use this function for rapid testing:


In [7]:
# Quick function to get advice - EDIT THE VALUES BELOW!
def quick_advice(age, income, expenses, debt, savings, question):
    """Get quick advice with minimal setup."""
    
    profile = ClientProfile(
        age=age,
        income=IncomeInfo(annual_salary=income),
        expenses=ExpensesInfo(
            housing=expenses * 0.3,
            food=expenses * 0.15,
            transportation=expenses * 0.1,
            other=expenses * 0.45,
        ),
        debts=DebtInfo(credit_card=debt) if debt > 0 else DebtInfo(),
        assets=AssetsInfo(checking_savings=savings),
    )
    
    scenario = FinancialScenario(
        client_profile=profile,
        question=question,
    )
    
    advisor = CFPAdvisor(
        model="gpt-4o-mini",
        temperature=0.7,
        api_key=os.getenv("OPENAI_API_KEY"),
    )
    
    advice = advisor.provide_advice(scenario)
    display(Markdown(f"## Advice\n\n{advice.summary}"))
    return advice

# Example - EDIT THESE VALUES AND RUN!
result = quick_advice(
    age=30,
    income=80000,
    expenses=3000,
    debt=5000,
    savings=10000,
    question="Should I invest in a Roth IRA or Traditional IRA?"
)


## Advice

### 1. PRIORITY ACTIONS

**1. Pay Down Credit Card Debt: $5,000**  
**2. Contribute to an Emergency Fund: $5,000**  
**3. Max Out Roth IRA Contribution: $6,500 (for 2023)**

### 2. DETAILED REASONING

**1. Pay Down Credit Card Debt: $5,000**  
Credit card debt typically carries high-interest rates (often 15% to 25%). By paying off the entire $5,000 balance, you eliminate interest payments that can significantly hinder your ability to build wealth in the long run. This action frees up cash flow for other financial goals and improves your credit score, making future borrowing more affordable.

**2. Contribute to an Emergency Fund: $5,000**  
While you currently have $10,000 in savings, it’s prudent to ensure that your emergency fund covers at least three to six months of living expenses ($9,000 to $18,000 for your monthly expenses of $3,000). Adding $5,000 will bring your total to $15,000, providing a strong financial cushion against unforeseen circumstances—such as job loss or unexpected expenses.

**3. Max Out Roth IRA Contribution: $6,500 (for 2023)**  
A Roth IRA allows your investments to grow tax-free, and withdrawals in retirement are also tax-free, provided certain conditions are met. Given your age and income level, contributing to a Roth IRA can maximize your long-term wealth-building potential. Investing early allows you to take advantage of compound growth, which is particularly beneficial when starting at a younger age.

### 3. TIMELINE

**Month 1**:  
- Pay off the credit card debt ($5,000).  
- Expect to see improved cash flow immediately, as you will eliminate monthly interest payments.

**Month 2-3**:  
- Allocate $5,000 to your emergency fund, bringing your total to $15,000.  
- This should be completed within three months.

**Month 4**:  
- Contribute $6,500 to your Roth IRA for the 2023 tax year.  
- Automate contributions if possible to ensure consistent investment.

### 4. EXPECTED OUTCOMES

- **Debt Elimination**: By paying off the credit card, you save an estimated $800-$1,250 in interest over the next year (assuming a 16% interest rate).
- **Emergency Fund**: A fully funded emergency fund of $15,000 will cover five months of expenses, providing peace of mind and financial stability.
- **Retirement Savings Growth**: Contributing to a Roth IRA can yield significant tax-free growth. If invested wisely, a $6,500 contribution could grow to over $100,000 in 30 years, assuming a conservative 7% annual return.

### 5. NEXT STEPS AND FOLLOW-UP RECOMMENDATIONS

- **Immediately**: Make the $5,000 payment on your credit card.  
- **Within the Next 2-3 Months**: Deposit an additional $5,000 into your emergency fund and ensure that you have set up a separate savings account for this purpose.
- **In Month 4**: Open a Roth IRA if you don’t already have one, and invest your $6,500 contribution. Consider a diversified portfolio that balances growth and risk suitable for your age and risk tolerance.
- **Follow-Up**: Schedule a follow-up meeting in three months to review your progress, assess your investment strategy in the Roth IRA, and discuss any changes in your financial goals or situation. 

By addressing your credit card debt first, solidifying your emergency fund, and investing in a Roth IRA, you create a solid foundation for future financial growth and stability.